In [4]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import tqdm
import time
import datetime
import pandas as pd
import json

# seven eleven


In [ ]:
start_url = "https://www.7-eleven.co.kr/util/storeLayerPop.asp"

# 각 시의 행정구역 데이터 호출을 위한 url, payload
get_gugun_url = "https://www.7-eleven.co.kr/library/asp/StoreGetGugun.asp"
get_gugun_payload = {
    'Sido' : None,
    'selName' : 'storeLayGu',    
}

# 각 시,군 별 세븐일레븐 편의점 데이터 호출을 위한 url, payload
get_store_url = "https://www.7-eleven.co.kr/util/storeLayerPop.asp"
"https://www.7-eleven.co.kr/util/storeLayerPop.asp"
get_store_payload = {
    'storeLaySido':None,
    'storeLayGu':None,
    'hiddentext':'none',
}

# 각 시를 불러오기 위한 요청
r = requests.get(start_url)
bs = BS(r.text)

# 데이터를 저장할 리스트
seven_store_data = []
for sido in tqdm.tqdm(bs.find(id = 'storeLaySido').findAll('option')[1:]) :
    
    # 각 페이로드에 sido값 지정
    get_gugun_payload['Sido'] = sido.text
    get_store_payload['storeLaySido'] = sido.text
    # 구,군 정보 호출
    gugun_r = requests.post(get_gugun_url, data = get_gugun_payload)
    for gugun in BS(gugun_r.text).findAll('option')[1:] :
        get_store_payload['storeLayGu'] = gugun.text
        get_store_r = requests.post(get_store_url, data = get_store_payload)
        
        for store in BS(get_store_r.text).find('div', {'class' : 'list_stroe'}).findAll('li') :
            try : 
                store_name = store.findAll('span')[0].text.strip()
                tmp = store.findAll('span')[1].text.split()
                # 'javascript: document.mapifrmAction.markerClick(1,37.564153483,127.00835317);' <- 문자열에서 위도 경도를 분해함
                lat, long = map(lambda x : float(x), store.find('a')['href'].split('(')[1].split(')')[0].split(',')[1:]) 
                store_addr = ''
                for text in tmp :
                    store_addr += text + ' '
                row = {
                    'brand' : 'seven',
                    'store_name' : store_name,
                    'store_addr' : store_addr,
                    'lat' : lat,
                    'long' : long,
                }
                
                seven_store_data.append(row)
            except :
                print(get_store_payload['storeLayGu'])

 18%|██████████████▋                                                                    | 3/17 [00:59<04:42, 20.20s/it]

# Emart

In [105]:
sido_list = [
    '강원특별자치도',
    '경기도',
    '경상남도',
    '경상북도',
    '광주광역시',
    '대구광역시',
    '대전광역시',
    '부산광역시',
    '서울특별시',
    '세종특별자치시',
    '울산광역시',
    '인천광역시',
    '전라남도',
    '전라북도',
    '제주특별자치도',
    '충청남도',
    '충청북도',
]

# 이마트 데이터 처리를 위한 함수
def select(x) :
    # Title
    try :
        title = x['TITLE']
    except :
        title = None

    # Address
    try :
        addr = x['ADDRESS']
    except :
        addr = None

    try :
        lat = x['LATITUDE']
    except :
        lat = None

    try :
        long = x['LONGITUDE']
    except :
        long = None
        
    row = {
        'brend' : 'emart',
        'store_name' : title,
        'store_addr' : addr,
        'lat' : lat,
        'long' : long,
    }
    return row

emart_store_data = []
for sido in sido_list :
    page = 0
    while True :
        page += 1
        url = f"https://www.emart24.co.kr/api1/store?page={page}&search=&AREA1={sido}&AREA2=&SVR_24=&SVR_AUTO=&SVR_PARCEL=&SVR_ATM=&SVR_WINE=&SVR_COFFEE=&SVR_SMOOTH=&SVR_APPLE=&SVR_TOTO="
        r = requests.get(url)#, params = payload)
        try :
            result = list(map(lambda x : select(x), r.json()['data']))
            emart_store_data += result
            if r.json()['count'] / 40 < page :
                break
        except :
            pass